#### Import numpy 

In [ ]:
import numpy as np
from matplotlib import pyplot
import MNISTtools

xtrain, ltrain = MNISTtools.load(dataset='training', path = '/datasets/MNIST')
print(xtrain.shape)
print(ltrain.shape)
MNISTtools.show(xtrain[:,42])

#### Helper Functions

In [ ]:
def normalize_MNIST_images(x):
    x = x.astype(np.float64)
    x = (x - 127.5)/127.5
    return x

xtrain = normalize_MNIST_images(xtrain)

def label2onehot(lbl):
    d = np.zeros((lbl.max() + 1, lbl.size))
    d[lbl[np.arange(0, lbl.size)], np.arange(0, lbl.size)] = 1
    return d

dtrain = label2onehot(ltrain)

def onehot2label(d):
    lbl = d.argmax(axis=0)
    return lbl

def softmax(a):
    M = a.max(axis = 0)
    func = np.exp(a - M)/np.sum(np.exp(a - M), axis = 0)
    return func

def softmaxp(a, e):
    s = softmax(a)
    x = np.multiply(s,e)
    y = np.multiply(np.sum(x, axis = 0),s)
    delta = x - y
    return delta

def relu(a):
    return np.where(a <= 0, 0, a)

def relup(a, e):
    r = relu(a)
    X1 = np.where(r == 0, 0,1)
    delta = np.multiply(X1,e)
    return delta

#### Verify the implementation of softmaxp and relup

In [ ]:
eps= 1e-6
a= np.random.randn(10, 200)
e= np.random.randn(10, 200)
diff= softmaxp(a, e)
diff_approx = (softmax(a + eps*e) - softmax(a))/eps
rel_error= np.abs(diff - diff_approx).mean() / np.abs(diff_approx).mean()
print(rel_error, ' should be smaller than 1e-6 ' )


eps= 1e-6
a= np.random.randn(10, 200)
e= np.random.randn(10, 200)
diff= relup(a, e)
diff_approx = (relu(a + eps*e) - relu(a))/eps
rel_error= np.abs(diff - diff_approx).mean() / np.abs(diff_approx).mean()
print(rel_error, ' should be smaller than 1e-6 ' )


#### Functions for initialization, forward prop , cross entropy loss and performance evaluation

In [ ]:
def init_shallow(Ni, Nh, No):
    b1 = np.random.randn(Nh, 1) / np.sqrt((Ni+1.)/2.)
    W1 = np.random.randn(Nh, Ni) / np.sqrt((Ni+1.)/2.)
    b2 = np.random.randn(No, 1) / np.sqrt((Nh+1.))
    W2 = np.random.randn(No, Nh) / np.sqrt((Nh+1.))
    return W1, b1, W2, b2

Ni = xtrain.shape[0]
Nh = 64
No = dtrain.shape[0]
netinit = init_shallow(Ni, Nh, No)

def forwardprop_shallow(x, net):
    W1 = net[0]
    b1 = net[1]
    W2 = net[2]
    b2 = net[3]
    a1 = W1.dot(x) + b1
    h1 = relu(a1)
    a2 = W2.dot(h1) + b2
    y = softmax(a2)
    return y

def eval_loss(y, d):
    return -np.mean((d*np.log(y)))

def eval_perfs(y, lbl):
    d = np.zeros((1, lbl.size))
    return 100*(np.sum(onehot2label(y[:,np.arange(0,lbl.size)]) != lbl[np.arange(0, lbl.size)]).astype(np.float64)/lbl.size)

In [ ]:
yinit = forwardprop_shallow(xtrain, netinit)
print(eval_loss(yinit, dtrain), ' should be around .26 ' )
print(eval_perfs(yinit, ltrain))

#### backprop implementation

In [ ]:
def update_shallow(x, d, net, gamma=.05):
    W1 = net[0]
    b1 = net[1]
    W2 = net[2]
    b2 = net[3]
    Ni = W1.shape[1]
    Nh = W1.shape[0]
    No = W2.shape[0]
    gamma = gamma / x.shape[1] # normalized by the training dataset size
        
    a1 = W1.dot(x) + b1
    z1 = relu(a1)
    a2 = W2.dot(z1) + b2
    y = softmax(a2)
    
    dw2 = softmaxp(a2,-1*(d/y))
    dw1 = relup(a1,(np.dot(W2.T,dw2)))
    
    W2 -= np.dot(dw2,z1.T)*gamma
    W1 -= np.dot(dw1, x.T)*gamma
    b2 -= np.sum(dw2,axis=1, keepdims=True)*gamma
    b1 -= np.sum(dw1,axis=1, keepdims=True)*gamma
        
    return W1, b1, W2, b2

def backprop_shallow(x, d, net, T, gamma=.05):
    lbl = onehot2label(d)
    for t in range(0, T):
        net = update_shallow(x,d,net,gamma)
        y = forwardprop_shallow(x, net)
        print('Loss is', eval_loss(y, dtrain))
        print('Miss-Clasifications', eval_perfs(y,lbl))
    return net

In [ ]:
nettrain = backprop_shallow(xtrain, dtrain, netinit, 20)

#### Prediction on test dataset 

In [ ]:
xtest, ltest = MNISTtools.load(dataset='testing', path = '/datasets/MNIST')
print(xtest.shape)
print(ltest.shape)

xtest = normalize_MNIST_images(xtest)
dtest = label2onehot(ltest)

y_predict = forwardprop_shallow(xtest, nettrain)
print('Test Loss is', eval_loss(y_predict, dtest))
print('Test Miss-Clasifications is', eval_perfs(y_predict,ltest))

#### Minibatch 

In [ ]:
def backprop_minibatch_shallow(x, d, net, T, B=100, gamma=.05):
    N = x.shape[1]
    lbl = onehot2label(d)
    for t in range(0, T):
        for l in range(0, (N+B-1)/B):
            idx = np.arange(B*l,min(B*(l+1), N))
            net = update_shallow(x[:,idx] , d[:,idx], net , gamma)
        y = forwardprop_shallow(x, net)
        print('Loss is', eval_loss(y, dtrain))
        print('Miss-Clasifications', eval_perfs(y,lbl))
    return net

Ni = xtrain.shape[0]
Nh = 64
No = dtrain.shape[0]
netinit = init_shallow(Ni, Nh, No)

netminibatch = backprop_minibatch_shallow(xtrain, dtrain, netinit, 5, B=100)

#### Prediction on Test Dataset

In [ ]:
y_predict = forwardprop_shallow(xtest, netminibatch)
print('Test Loss is', eval_loss(y_predict, dtest))
print('Test Miss-Clasifications is', eval_perfs(y_predict,ltest))